In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install -qU "langchain[google-genai]"
%pip install -qU faiss-cpu
%pip install -qU pypdf
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.7 MB/s eta 0:00:00
ERROR: pip's dependen

# **IMPORT PACKAGES**

In [2]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Chat Model

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

# Embeddings Model

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

## Embeddings example

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "A swift brown fox leaps over a lethargic canine."

# Embed the sentences
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)

In [36]:
# Convert embeddings to numpy arrays for calculation
embedding1_np = np.array(embedding1).reshape(1, -1)
embedding2_np = np.array(embedding2).reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding1_np, embedding2_np)[0][0]

print(f"Sentence 1: {sentence1}")
print(f"Sentence 2: {sentence2}\n")
print(f"Cosine similarity between the embeddings: {similarity_score}")

Sentence 1: The quick brown fox jumps over the lazy dog.
Sentence 2: A swift brown fox leaps over a lethargic canine.

Cosine similarity between the embeddings: 0.8499936585079755


In [5]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Vector Store/Database

In [6]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
print(embedding_dim)
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

3072


In [7]:
# Ensure embedding_dim is defined (from your embeddings model)
# embedding_dim = len(embeddings.embed_query("hello world"))

# Initialize the HNSW index
# M: The number of neighbors each vertex has in the graph on each layer
# ef_construction: This parameter controls the trade-off between construction time and index quality.
M = 16
ef_construction = 200
index_hnsw = faiss.IndexHNSWFlat(embedding_dim, M)
index_hnsw.hnsw.efConstruction = ef_construction

# Create the new FAISS vector store with the HNSW index
vector_store_hnsw = FAISS(
    embedding_function=embeddings,
    index=index_hnsw,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

print("HNSW Vector Store created successfully.")

HNSW Vector Store created successfully.


In [8]:
embeddings.embed_query("hello world")[:5]

[-0.030033651739358902,
 0.004021478816866875,
 0.01552528329193592,
 -0.08638038486242294,
 -0.0035104958806186914]

# Indexing

## Loading documents

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader

# Define a list of PDF file paths
pdf_paths = [
    "/content/Wells Fargo Active Cash Credit Card Credit Terms and Conditions.pdf",
    "/content/Bilt World Elite Mastercard Credit Card Terms and Conditions.pdf"
]

all_pages = []
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pages = []
    # Use alazy_load for efficient loading of large documents
    async for page in loader.alazy_load():
        pages.append(page)
    all_pages.extend(pages)

total_characters = sum(len(page.page_content) for page in all_pages)
print(f"Total characters: {total_characters} \nTotal pages: {len(all_pages)}")

Total characters: 59975 
Total pages: 24


In [11]:
print(f"{all_pages[0].metadata}\n")
print(all_pages[1].page_content)

{'producer': 'cairo 1.18.0 (https://cairographics.org)', 'creator': 'Mozilla Firefox 143.0.4', 'creationdate': '2025-10-09T13:54:38+05:30', 'source': '/content/Wells Fargo Active Cash Credit Card Credit Terms and Conditions.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}

ATMs/Locations
Customer Service and FAQs
Español
Wells Fargo Active Cash  Credit Card Terms and Conditions
Wells Fargo Active Cash Visa  Card Terms and Conditions
Thank you for your interest in opening a Wells Fargo Active Cash Account.
You may not be eligible for introductory annual percentage rates, fees and/or rewards bonus o�ers if you
have the Wells Fargo Active Cash Visa or opened one within the last 48 months from the date of this application -
even if that account is closed and has a $0 balance.
IMPORTANT DISCLOSURES
Interest Rates and Interest Charges
Annual Percentage Rate (APR)
for Purchases
 0% introductory APR for 12 months from date of account opening.
After that, your APR will be 18.99%, 23.99%, 

## Splitting documents

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(all_pages)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 81 sub-documents.


In [13]:
all_splits[1].page_content

'ATMs/Locations\nCustomer Service and FAQs\nEspañol\nWells Fargo Active Cash  Credit Card Terms and Conditions\nWells Fargo Active Cash Visa  Card Terms and Conditions\nThank you for your interest in opening a Wells Fargo Active Cash Account.\nYou may not be eligible for introductory annual percentage rates, fees and/or rewards bonus o�ers if you\nhave the Wells Fargo Active Cash Visa or opened one within the last 48 months from the date of this application -\neven if that account is closed and has a $0 balance.\nIMPORTANT DISCLOSURES\nInterest Rates and Interest Charges\nAnnual Percentage Rate (APR)\nfor Purchases\n\xa00% introductory APR for 12 months from date of account opening.\nAfter that, your APR will be 18.99%, 23.99%, or 28.99%,\nbased on your creditworthiness. This APR will vary with the market based\non the U.S. Prime Rate.\nAnnual Percentage Rate (APR)\nfor Balance Transfers\n\xa00% introductory APR for 12 months from date of account opening.'

## Embeddinbg the documents and Storing documents

Now we need to index our 81 text chunks so that we can search over them at runtime.
To search the content of the document semantically, we will embed the contents of each document split and insert these embeddings into a vector store. Given an input query, we can then use vector search to retrieve relevant documents.

In [14]:
document_ids = vector_store.add_documents(documents=all_splits)

print("index IDs: ",document_ids[:3])

index IDs:  ['115382dc-3c46-4e43-a4df-746c4e804fa1', 'de55426c-d448-44be-b090-dcd318079f66', 'e7dd8010-3336-4abc-baa0-b7f6677f64f1']


In [15]:
# document_ids_HNSW = vector_store_hnsw.add_documents(documents=all_splits)
# print("index IDs: ",document_ids_HNSW[:3])

# Retrieval

In [16]:
question = "what are the Annual Percentage Rate (APR) for Purchases on Active cash card of Wells Fargo?"

In [17]:
retrieved_docs = vector_store.similarity_search(question)
# retrieved_docs = vector_store_hnsw.similarity_search(question)

In [18]:
print(len(retrieved_docs))
# for i in retrieved_docs:
#   print(i.page_content)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
docs_content

4


'ATMs/Locations\nCustomer Service and FAQs\nEspañol\nWells Fargo Active Cash  Credit Card Terms and Conditions\nWells Fargo Active Cash Visa  Card Terms and Conditions\nThank you for your interest in opening a Wells Fargo Active Cash Account.\nYou may not be eligible for introductory annual percentage rates, fees and/or rewards bonus o�ers if you\nhave the Wells Fargo Active Cash Visa or opened one within the last 48 months from the date of this application -\neven if that account is closed and has a $0 balance.\nIMPORTANT DISCLOSURES\nInterest Rates and Interest Charges\nAnnual Percentage Rate (APR)\nfor Purchases\n\xa00% introductory APR for 12 months from date of account opening.\nAfter that, your APR will be 18.99%, 23.99%, or 28.99%,\nbased on your creditworthiness. This APR will vary with the market based\non the U.S. Prime Rate.\nAnnual Percentage Rate (APR)\nfor Balance Transfers\n\xa00% introductory APR for 12 months from date of account opening.\n\nthree business days prior t

# Augmentation

In [19]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

messages = prompt.invoke(
    {"context": docs_content, "question": question}
).to_messages()

print(messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: what are the Annual Percentage Rate (APR) for Purchases on Active cash card of Wells Fargo? 
Context: ATMs/Locations
Customer Service and FAQs
Español
Wells Fargo Active Cash  Credit Card Terms and Conditions
Wells Fargo Active Cash Visa  Card Terms and Conditions
Thank you for your interest in opening a Wells Fargo Active Cash Account.
You may not be eligible for introductory annual percentage rates, fees and/or rewards bonus o�ers if you
have the Wells Fargo Active Cash Visa or opened one within the last 48 months from the date of this application -
even if that account is closed and has a $0 balance.
IMPORTANT DISCLOSURES
Interest Rates and Interest Charges
Annual Percentage Rate (APR)
for Purchases
 0% introductory APR for 12 months from 

# Generation

In [20]:
question = "what are the Annual Percentage Rate (APR) for Purchases on Active cash card?"

In [21]:
answer = llm.invoke(question)

In [22]:
from IPython.display import Markdown, display

display(Markdown(answer.content))

The Annual Percentage Rate (APR) for Purchases on the Wells Fargo Active Cash® Card is typically a **variable rate** that depends on your creditworthiness.

As of my last update, the variable APR for purchases on the Wells Fargo Active Cash card generally ranges from approximately **19.24% to 29.24%**.

**Important Considerations:**

*   **Variable Rate:** This means the APR can change with the market Prime Rate.
*   **Creditworthiness:** Your specific APR within that range will be determined by Wells Fargo based on your credit history when you apply.
*   **Introductory APR:** The Active Cash card often features an introductory 0% APR on purchases for a certain period (e.g., 15 months from account opening). After this introductory period, the standard variable APR applies to any remaining balance.
*   **Other APRs:** There are also separate APRs for balance transfers and cash advances, which may differ from the purchase APR.

**Always check the most current and official terms and conditions on the Wells Fargo website or in your cardholder agreement for the most accurate and up-to-date information.**

In [23]:
answer2 = llm.invoke(messages)

In [24]:
from IPython.display import Markdown, display

display(Markdown(answer2.content))

The Wells Fargo Active Cash card offers a 0% introductory APR for purchases for 12 months from the account opening date. After this period, the APR will be 18.99%, 23.99%, or 28.99%, based on creditworthiness and will vary with the market based on the U.S. Prime Rate. The maximum APR for purchases is 29.99%.

In [25]:
from langchain import hub

def retrieve_and_augment(query, vector_store):
  """
  Retrieves similar context from the vector store and augments with the RAG prompt.

  Args:
    query: The user's query string.
    vector_store: The FAISS vector store object.

  Returns:
    A list of messages formatted with the RAG prompt and retrieved context.
  """
  retrieved_docs = vector_store.similarity_search(query)
  docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

  prompt = hub.pull("rlm/rag-prompt")

  messages = prompt.invoke(
      {"context": docs_content, "question": query}
  ).to_messages()

  return messages

In [37]:
question2 = "can you compare both active cash vard and blit world elite card on APR? generate a output table to show the comparision"
messages2 = retrieve_and_augment(question2, vector_store)

In [38]:
messages2

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: can you compare both active cash vard and blit world elite card on APR? generate a output table to show the comparision \nContext: Customer Service and FAQs\nEspañol\nBilt World Elite Mastercard Credit Card Terms and Conditions\nBilt World Elite Mastercard  Credit Card Terms and Conditions\nIssued and administered by Wells Fargo Bank, N.A., pursuant to a license from Mastercard International.\nIMPORTANT DISCLOSURES\n  Interest Rates and Interest Charges\nAnnual Percentage Rate (APR) for\nPurchases\n20.24%, 23.24%, or 28.24%, based on your creditworthiness.\nThis APR will vary with the market based on the U.S. Prime Rate.\nAnnual Percentage Rate (APR) for\nBalance Transfers\n20.24%, 23.24%, or 28.24%, based on your cred

In [40]:
answer3 = llm.invoke(messages2)

In [41]:
display(Markdown(answer3.content))

Here's a comparison of the APRs for the Bilt World Elite Mastercard and the Active Cash Card:

| APR Type          | Bilt World Elite Mastercard                 | Active Cash Card                                          |
| :---------------- | :------------------------------------------ | :-------------------------------------------------------- |
| **Purchases**     | 20.24%, 23.24%, or 28.24%                   | 18.99%, 23.99%, or 28.99%                                 |
| **Balance Transfers** | 20.24%, 23.24%, or 28.24%                   | 0% intro for 12 months, then 18.99%, 23.99%, or 28.99% |
| **Cash Advances** | 27.99%                                      | 29.99%                                                    |

The Active Cash Card offers a 0% introductory APR for 12 months on balance transfers, after which the rate adjusts to 18.99%, 23.99%, or 28.99%. All APRs for both cards are variable and based on creditworthiness and the U.S. Prime Rate.